# Crawl Data

In [6]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import csv
import re

In [ ]:
class crawl_Data:
    def __init__(self, ):
        browser = webdriver.Chrome(executable_path='driver\chromedriver.exe')
        path_search = ""


## Foody

In [3]:
def preprocess(text):
    """Preprocess data
    Args:
        text:

    Returns:

    """
    html_pattern = re.compile('<.*?>')
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = html_pattern.sub(r' ', text)
    text = url_pattern.sub(r' ', text)
    text = text.replace("\n", ". ")
    text = text.replace(";", "")

    return text

In [8]:
class Foody(crawl_Data):
    def Foody(self, path_search):
        self.path_search = path_search
        self.browser.close()


browser = webdriver.Chrome(executable_path='driver\chromedriver.exe')
# Maximize window
browser.maximize_window()
# Open foody
path_search = "https://www.foody.vn/binh-dinh/travel/khu-du-lich"
browser.get(path_search)
sleep(2)
# Scroll page foody to get more location
for _ in range(10):
    browser.execute_script("window.scrollBy(0, 600)")
    sleep(2)
# Get number comments and links
cmts = browser.find_elements(By.XPATH, "//span[@data-bind='text: TotalReview.formatK(1)']")
elements = browser.find_elements(By.XPATH, "//div[@class='resname']//h2//a")

location = []
for i in range(len(cmts)):
    if int(cmts[i].text) >= 2:
        location.append(elements[i].get_attribute("href"))

for link in location:
    browser.get(link)
    sleep(2)

    browser.execute_script("window.scrollBy(0, document.body.scrollHeight)")
    sleep(2)

    reviews = browser.find_elements(By.XPATH, "//span[@ng-bind-html='Model.Description']")
    print("total reviews", len(reviews))

    results = []
    for review in reviews:
        text = review.text
        text = preprocess(text)
        if len(text) < 40:
            continue
        results.append([text])

    print("total results", len(results))

    with open(f"data_foody.csv", "a", encoding='utf-8', newline='') as file:
        # Create a CSV writer
        writer = csv.writer(file)

        # Write data to the file
        writer.writerows(results)

    sleep(2)

# Close browser
browser.close()

C:\Users\hongt\AppData\Local\Temp\ipykernel_11016\3443980191.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='driver\chromedriver.exe')


total reviews 22
total results 22
total reviews 20
total results 20
total reviews 16
total results 16
total reviews 12
total results 12
total reviews 12
total results 12
total reviews 6
total results 6
total reviews 5
total results 5
total reviews 2
total results 2
total reviews 5
total results 5
total reviews 3
total results 3
total reviews 3
total results 3
total reviews 2
total results 1
total reviews 2
total results 2
total reviews 2
total results 2
total reviews 2
total results 2


## Google Map

In [ ]:
browser = webdriver.Chrome(executable_path=r'./driver/chromedriver.exe')

# Maximize window
browser.maximize_window()

# Open google map
path_search = "https://www.google.com/maps/@9.779349,105.6189045,11z?hl=vi-VN"
browser.get(path_search)
sleep(2)

# Search
input_search = browser.find_element(By.ID,"searchboxinput")
input_search.send_keys("nhà hàng vũng tàu")
input_search.send_keys(Keys.ENTER)
sleep(4)

def preprocess(text):
    html_pattern = re.compile('<.*?>')
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = html_pattern.sub(r' ', text)
    text = url_pattern.sub(r' ', text)
    text = text.replace("\n", ".")
    text = text.replace(";", "")

    return text

scroll_items = browser.find_elements(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf ecceSd']")
sleep(3)
for _ in range(50):
    sleep(2)
    scroll_items[1].send_keys(Keys.DOWN);

# Get result search
output_searchs = browser.find_elements(By.XPATH, "//div[@class='Nv2PK THOPZb CpccDe']")

print(len(output_searchs))
sleep(3)

for output_search in output_searchs:
    results = []

    try:
        sleep(1)
        scroll_items[1].send_keys(Keys.DOWN)

        index = output_searchs.index(output_search)
        print("Item: ", index)

        # Choose each output
        output_search.click()
        sleep(3)

        # Get label review and click
        label_review = browser.find_element(By.XPATH, "//button[@class='DkEaL']")
        numbers_review = label_review.text.replace(".", "")
        numbers_review = int(re.findall(r'\d+', numbers_review)[0])
        print("scroll", numbers_review)
        if numbers_review > 1000:
            numbers_review = 1000

        label_review.click()
        sleep(3)
    except:
        print("Error")
        continue

    # Scroll to get more result
    scroll = browser.find_element(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf']")
    for _ in range(numbers_review):
        # Click "Xem thêm" to get full review
        try:
            see_more = browser.find_element(By.XPATH, "//button[@aria-label='Xem thêm']")
            see_more.click()
        except:
            pass

        # Scroll
        sleep(0.5)
        scroll.send_keys(Keys.DOWN);

    # Get reviews
    reviews = browser.find_elements(By.XPATH, "//span[@class='wiI7pd']")
    print("total reviews", len(reviews))

    # Preproces review
    for review in reviews:
        text = review.text
        text = preprocess(text)
        if len(text) < 40:
            continue
        results.append([text])

    print("total results", len(results))

    # Write data to file csv
    with open(f"../datasets/data_crawl/data_vungtau_ggmap.csv", "a", encoding='utf-8', newline='') as file:
        # Create a CSV writer
        writer = csv.writer(file)

        # Write data
        writer.writerows(results)

    sleep(3)
# Đóng trình duyệt
browser.close()

## Google Map 1 Item

In [ ]:
browser = webdriver.Edge(executable_path='../driver\msedgedriver.exe')

# Maximize window
browser.maximize_window()

# Open google map
path_search = "https://www.google.com/maps/@9.779349,105.6189045,11z?hl=vi-VN"
browser.get(path_search)
sleep(2)

# Open google map
path_search = "https://goo.gl/maps/fGn2vCcsAPZKTa5G7"

browser.get(path_search)
sleep(2)

def preprocess(text):
    html_pattern = re.compile('<.*?>')
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = html_pattern.sub(r' ', text)
    text = url_pattern.sub(r' ', text)
    text = text.replace("\n", ".")
    text = text.replace(";", "")

    return text

results = []

sleep(1)

# Get label review and click
label_review = browser.find_element(By.XPATH, "//button[@class='DkEaL']")
numbers_review = label_review.text.replace(".", "")
numbers_review = int(re.findall(r'\d+', numbers_review)[0])
print("scroll", numbers_review)

if numbers_review > 500:
    numbers_review = 500

label_review.click()
sleep(3)

# Scroll to get more result
scroll = browser.find_element(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf']")
for _ in range(numbers_review):
    # Click "Xem thêm" to get full review
    try:
        see_more = browser.find_element(By.XPATH, "//button[@aria-label='Xem thêm']")
        see_more.click()
    except:
        pass

    # Scroll
    sleep(0.5)
    scroll.send_keys(Keys.DOWN);

# Get reviews
reviews = browser.find_elements(By.XPATH, "//span[@class='wiI7pd']")
print("total reviews", len(reviews))

# Preproces review
for review in reviews:
    text = review.text
    text = preprocess(text)
    if len(text) < 30:
        continue
    results.append([text])

print("total results", len(results))

# Write data to file csv
with open(f"../datasets/data_crawl/data_taxi_ggmap.csv", "a", encoding='utf-8', newline='') as file:
    # Create a CSV writer
    writer = csv.writer(file)

    # Write data
    writer.writerows(results)

sleep(3)
# Đóng trình duyệt
browser.close()